importing find spark and pyspark

In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

start sparksession

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

importing row types

In [3]:
from pyspark.sql import Row

adding the schema to the RDD to create dataFrame

In [4]:
sc=spark.sparkContext

creating a dataframe from sorce .csv file

In [5]:
customerDF=spark.read.load("/home/hduser/Downloads/sharedfolder/customers.tsv",format="csv",sep="\t",inferschema="true")
salestxnsDF=spark.read.load("/home/hduser/Downloads/sharedfolder/salestxns.tsv",format="csv",sep="\t",inferschema="true")

Different types of operations on dataFrame

In [6]:
customerDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)



In [7]:
salestxnsDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)



In [8]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [9]:
new_names_sDF = ['salestxn_id','category_id','category_name','product_id','product_name','product_price','product_quantity','customer_id']

In [10]:
cDF2 = customerDF.toDF(*new_names_cDF)

In [11]:
sDF2 = salestxnsDF.toDF(*new_names_sDF)

In [12]:
cDF2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [13]:
cDF2.show(5)

+-----------+--------------+-------------+--------------+----------------+
|customer_id| customer_name|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-------------+--------------+----------------+
|      11039|   Mary Torres|       Caguas|            PR|             725|
|       5623|    Jose Haley|     Columbus|            OH|           43207|
|       5829|    Mary Smith|      Houston|            TX|           77015|
|       6336|Richard Maddox|       Caguas|            PR|             725|
|       1708|Margaret Booth|    Arlington|            TX|           76010|
+-----------+--------------+-------------+--------------+----------------+
only showing top 5 rows



In [14]:
sDF2.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [15]:
sDF2.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+
|          1|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|      11599|
|          2|         48|    Water Sports|      1073|Pelican Sunstream...|       199.99|               1|        256|
|          3|         24| Women's Apparel|       502|Nike Men's Dri-FI...|         50.0|               5|        256|
|          4|         18|  Men's Footwear|       403|Nike Men's CJ Eli...|       129.99|               1|        256|
|          5|         40|     Accessories|       897|Team Golf New Eng...|        24.99|               2|       8827|
+-----------+-----------+----------------+----------+---

In [16]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [17]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:  +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv
+- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
   +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv

== Analyzed Logical Plan ==
customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int, salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _

In [18]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#52], [customer_id#70], Inner, BuildRight
:- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:  +- *Filter isnotnull(_c0#12)
:     +- *FileScan csv [_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/Downloads/sharedfolder/customers.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c0)], ReadSchema: struct<_c0:int,_c1:string,_c2:string,_c3:string,_c4:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[7, int, true] as bigint)))
   +- *Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
      +- *Filter isnotnull(_c7#42)
         +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,

We can notice that the execution plan is optimized in that customer is broad cast
now lets to it with small at the right side

In [19]:
jDF2=sDF2.join(cDF2,sDF2.customer_id==cDF2.customer_id)

In [20]:
type(jDF2)

pyspark.sql.dataframe.DataFrame

In [21]:
jDF2.explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#70], [customer_id#52], Inner, BuildRight
:- *Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- *Filter isnotnull(_c7#42)
:     +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/Downloads/sharedfolder/salestxns.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c7)], ReadSchema: struct<_c0:int,_c1:int,_c2:string,_c3:int,_c4:string,_c5:double,_c6:int,_c7:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
   +- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- *Filter isnotnull(_c0#12)
         +- *File

In [22]:
jDF2.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
   +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2

In [23]:
jDF3 = sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [24]:
jDF3 = sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id)

In [25]:
jDF3.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [26]:
jDF3.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1

importing pyspark.sql.functions

In [27]:
pyspark.sql.functions.broadcast(cDF2)

DataFrame[customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int]

In [28]:
type(pyspark.sql.functions.broadcast(cDF2))

pyspark.sql.dataframe.DataFrame

In [29]:
jDF4=sDF2.join(pyspark.sql.functions.broadcast(cDF2),sDF2.customer_id==cDF2.customer_id)

In [30]:
jDF4.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [31]:
jDF4.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1